In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
from zoneinfo import ZoneInfo
import math

In [12]:
subject_id = "001"
experiment_id = "001"

viewing_distance_cm = 70       # モニタまでの距離
screen_width_cm = 47.6           # モニタの横幅
screen_height_cm = 26.8          # モニタの高さ
diag_length_cm = math.sqrt(screen_width_cm**2 + screen_height_cm**2)

In [13]:
pose1_df = pd.read_csv(f"./exported_csv/pose_df/pose1_df_id{subject_id}-{experiment_id}.csv")
pose2_df = pd.read_csv(f"./exported_csv/pose_df/pose2_df_id{subject_id}-{experiment_id}.csv")

In [14]:
from math import atan, sqrt


def evaluate_precision(df, pose_name):
    total_len = len(df)
    
    # validity_sum == 2 のデータのみを使う
    valid_df = df[df["validity_sum"] == 2].copy()

    std_x = valid_df["mean_x"].std()
    std_y = valid_df["mean_y"].std()
    
    precision_rms = sqrt(std_x**2 + std_y**2)
    
    precision_cm = precision_rms * diag_length_cm
    precision_deg = atan(precision_cm / viewing_distance_cm)



    # サマリデータの返却
    return {
        "subject_id": subject_id,
        "task_id": experiment_id,
        "pose": pose_name,
        "pose_length": total_len,
        "valid_data_count": len(valid_df),
        "precision_rms": precision_rms,
        "precision_cm": precision_cm,
        "precision_deg": precision_deg
    }

In [15]:
summary_list = []
summary_list.append(evaluate_precision(pose1_df, "pose1"))
summary_list.append(evaluate_precision(pose2_df, "pose2"))

In [16]:
summary_df = pd.DataFrame(summary_list)
print(summary_df)

  subject_id task_id   pose  pose_length  valid_data_count  precision_rms  \
0        001     001  pose1         4028              2784       0.099955   
1        001     001  pose2         4061              2531       0.103686   

   precision_cm  precision_deg  
0      5.460144       0.077844  
1      5.663977       0.080738  
